In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 8, 6
import glob
import sys
import pickle
from scipy import interpolate ,optimize
from scipy.interpolate import UnivariateSpline
from tqdm import tqdm_notebook as tqdm#from tqdm import tqdm
#look at fixing later 
import warnings
warnings.filterwarnings('ignore')

#import functions from file in path 
sys.path.insert(0, '/home/ben/Documents/Bonn-uni/internship/python_code/functions')
#from functions2 import MS_finder2
from data_loading import load_masses, load_models
#sys.path.insert(0, '/home/ben/Documents/Bonn-uni/internship/python_code/Interpolation')
from interpolation import M_V_T_interp, MVT_interp_setup, v_over_vcrit_interp

data_dir='/home/ben/Documents/Bonn-uni/internship/takeaway_data/' #'/users/bhastings/internship/data/MW/' # #

file_ext= '.dat2'

In [2]:
#a=pickle.load(open('crit_rot_dataM3-25.pk', 'rb'))
print np.unique(a.keys())
#print a[12.0]

for h in a.keys():
    print h , a[h]

NameError: name 'a' is not defined

In [ ]:
#a=pickle.load(open('crit_rot_dataM3-25.pk', 'rb'))

for M in (a.keys())[1:]:
    #print M

    crit_rots_df= a[M]
    
    v_grid= np.unique(crit_rots_df['V_init'])
    t_grid= np.unique(crit_rots_df['t'])
    
    for vi in np.unique(crit_rots_df['V_init']):
        di=crit_rots_df[crit_rots_df['V_init'] == vi ]
        x=di['t/T']
        y=di['V_init']

        ix=x.argmax()
        im= x.argmin()

        plt.plot(x, y, 'ks', markersize=3)
        
        
    plt.title(str(M)+'---'+str(0.99))
    plt.show()
            
            

In [ ]:
#v_crit_min_master=pickle.load(open('crit_rot_data2.pk', 'rb'))
print v_crit_min_master.keys()


for M in (np.flip(mass_list))[0:1]:#np.sort(v_crit_min_master[B].keys()):
    plt.clf()
    for B in np.sort(v_crit_min_master.keys()):
        
        if M in v_crit_min_master[B]:
            crit_rots_df=v_crit_min_master[B][M]

        else :
            continue
        #crit_rots_df= crit_rot_data[M]
        #print len(np.unique(crit_rots_df['t/T']))
        vcritmin_df=pd.DataFrame()
        for vi in np.unique(crit_rots_df['V_init']):
            di=crit_rots_df[crit_rots_df['V_init'] == vi ]
            x=di['t']
            y=di['V_init']

            #ix=x.argmax()
            #im= x.argmin()

            plt.plot(x/1e6, y, 'ks', markersize=2)
        
            for ti in np.unique(di['t']):
                plot_arr_df=plot_arr_df.append(pd.DataFrame([[ti,vi,1]], columns=['t','v','a']))
       
        plt.title(str(M) + str('$M_{\odot}$') +' $V/V_{crit}$ > '+ str(B))
        #plt.legend(loc='lower left')
        plt.xlabel('t/Myr')
        plt.ylabel('V/$kms^{-1}$')
        plt.savefig(str(M)+'-'+str(B)+'.png', dpi=300)
        #plt.show()

In [ ]:


data_files, mass_list= load_masses(data_dir, 25)

data_df=load_models(data_files)

vmax_dict={}
for m in mass_list:
    di=data_df[data_df.M == m ]
    vmax=np.unique(di['V']).max()
    #print di['V_init'][di['V']== vmax].iloc[0], m
    vmax_dict[m]=di['V_init'][di['V']== vmax].iloc[0]

In [ ]:
B=0.9
print str(int(100*B))

# Interpolate V-M-T # 

In [ ]:
def MVT_interp_setup(data_df, mass_list):
    df=data_df[data_df['1:t[s]']== 0]

    v_T_interp={}
    for m in mass_list: 
        x=np.array(df['V_init'][df.M==m])#[-20:]
        y=np.array(df['M'][df.M==m])#[-20:]
        z=np.array(df['T_MS'][df.M==m])#[-20:]

        func=interpolate.interp1d(x,z,bounds_error=False, assume_sorted=True)
        v_T_interp[m]=func
    return v_T_interp, mass_list

def M_V_T_interp(M, V, v_T_interp_obj):
    v_T_interp=v_T_interp_obj[0]
    mass_list= v_T_interp_obj[1]
    x= mass_list
    y=map((lambda u: v_T_interp[u](V)),x)#v_T_interp[m](v)

    func2=interpolate.interp1d(x,y,bounds_error=False, assume_sorted=True)

    return func2(M)

v_T_interp= MVT_interp_setup(data_df, mass_list)
print M_V_T_interp(4,450, v_T_interp)

In [ ]:
df=data_df[data_df['1:t[s]']== 0]

v_T_interp={}
for m in mass_list: 
    x=np.array(df['V_init'][df.M==m])#[-20:]
    y=np.array(df['M'][df.M==m])#[-20:]
    z=np.array(df['T_MS'][df.M==m])#[-20:]
    
    func=interpolate.interp1d(x,z,bounds_error=False, assume_sorted=True)
    v_T_interp[m]=func
    
def M_V_T_interp(M, V):
    x= mass_list
    y=map((lambda u: v_T_interp[u](V)),x)#v_T_interp[m](v)
    
    func=interpolate.interp1d(x,y,bounds_error=False, assume_sorted=True)
    
    return func(M)
   


In [ ]:
t=0
m_i=15
Be_value=0.7

    
plot_df=pd.DataFrame()
for Be_value in (0.6,0.7,0.8,0.9,0.99):
    plot_arr=np.array([])
    for m_i in [15]:#mass_list:
        sys.stdout.write ('\r'+ str(Be_value)+'--'+str(m_i))
        dm=data_df[data_df.M ==m_i]
        #v_crit_min_arr=np.array([[]])

        #for t in np.linspace(0,100, 100): # times in Myr 
        #    t=t*1e6
        dg=pd.DataFrame()
        for vi in np.unique(dm['V_init'])[1:]:
            di= dm[(dm['V_init']==vi) & (dm['t/Tms']<0.99)  ]
            ti=((di['V/Vcrit']-(Be_value)).abs()).argmin()
            
            if di['V/Vcrit'].iloc[ti]> 0.99*Be_value:
                dn= di.iloc[ti]
                dg=dg.append(dn)

            
        plt.plot(dg['1:t[s]']/1e6, dg['V_init'], 'ko')
        plt.show()
        straight_line1=interpolate.interp1d(dg['V_init'], dg['V/Vcrit'],
                                                   bounds_error=False, assume_sorted=True)
      

#print plot_df.to_string()
print 'DONE'
#plot_df.to_pickle('./plot_df.pk')

In [ ]:
print len(np.arange(0,0.99,0.002))
print np.linspace(0,0.99,496)

In [ ]:
def v_over_vcrit_interp2(m_i, vfrac, v_T_interp_obj, plot):
    '''takes a m_i on the grid and a fraction of critical velocity v_init not on the grid and produces by 
    interpolation the v/vcrit against t or t/T curve'''
    v_T_interp=v_T_interp_obj[0]
    mass_list= v_T_interp_obj[1]
    plot_df=pd.DataFrame()
    dm=data_df[data_df.M ==m_i]
   

    for t_frac in np.linspace(0,0.99, 9):
        dg=pd.DataFrame()
        for vi in np.unique(dm['V/Vc_initial'])[1:]:
            di= dm[(dm['V/Vc_initial']==vi)  ]
            ti=((di['t/Tms']-(t_frac)).abs()).argmin()
            dn= di.iloc[ti]
            dg=dg.append(dn)

            
                #straight_line1=interpolate.interp1d( dg['V/Vcrit'],dg['V_init'],
                #                                       bounds_error=False, assume_sorted=True)
        straight_line1=interpolate.interp1d(dg['V/Vc_initial'], dg['V/Vcrit'],
                                                       bounds_error=True, assume_sorted=True)

        plt.plot(dg['V/Vc_initial'], dg['V/Vcrit'], 'o')
        plt.plot(vfrac,straight_line1(vfrac), 'ro' )
        plt.show()
        v_over_vcrit=straight_line1(vfrac)
        
        

        plot_df=plot_df.append(pd.DataFrame([[m_i,vfrac,t_frac,v_over_vcrit]], 
                                               columns=['M','Vfrac_init', 't/T', 'V/Vcrit']))

        
    plot_df.reset_index(drop=True, inplace=True)
    
    
    #interpolate MS lifetime of star 
    #T= M_V_T_interp(m_i, v_init, v_T_interp_obj)
    
    # insert real time into dataframe 
    #plot_df['t']= plot_df['t/T']* T
    
    #plot=False
    if plot== True:
        plt.plot( plot_df['t/T'],plot_df['V/Vcrit'], 'r-')
        plt.title(str(vfrac)+'   '+str(m_i))
        plt.xlabel('t/T')
        plt.ylabel('V/Vcrit')
        plt.show()

    return plot_df

s=v_over_vcrit_interp2(20, 0.15, v_T_interp, True)

In [ ]:

def v_over_vcrit_interp(m_i, v_init, v_T_interp_obj, plot):
    '''takes a m_i on the grid and a velocity v_init not on the grid and produces by 
    interpolation the v/vcrit against t or t/T curve'''
    v_T_interp=v_T_interp_obj[0]
    mass_list= v_T_interp_obj[1]
    plot_df=pd.DataFrame()
    dm=data_df[data_df.M ==m_i]
   

    for t_frac in np.linspace(0,0.99, 99):
        dg=pd.DataFrame()
        for vi in np.unique(dm['V_init'])[1:]:
            di= dm[(dm['V_init']==vi)  ]
            ti=((di['t/Tms']-(t_frac)).abs()).argmin()
            dn= di.iloc[ti]
            dg=dg.append(dn)

            
                #straight_line1=interpolate.interp1d( dg['V/Vcrit'],dg['V_init'],
                #                                       bounds_error=False, assume_sorted=True)
        straight_line1=interpolate.interp1d(dg['V_init'], dg['V/Vcrit'],
                                                       bounds_error=False, assume_sorted=True)

        v_over_vcrit=straight_line1(v_init)
        
        

        plot_df=plot_df.append(pd.DataFrame([[m_i,v_init,t_frac,v_over_vcrit]], 
                                               columns=['M','V_init', 't/T', 'V/Vcrit']))

        
    plot_df.reset_index(drop=True, inplace=True)
    
    
    #interpolate MS lifetime of star 
    T= M_V_T_interp(m_i, v_init, v_T_interp_obj)
    
    # insert real time into dataframe 
    plot_df['t']= plot_df['t/T']* T
    
    #plot=False
    if plot== True:
        plt.plot( plot_df['t/T'],plot_df['V/Vcrit'], 'r-')
        plt.title(str(v_init)+'   '+str(m_i))
        plt.xlabel('t/T')
        plt.ylabel('V/Vcrit')
        plt.show()

    return plot_df

s=v_over_vcrit_interp(20, 200, v_T_interp, True)


In [ ]:

plot_df=pd.read_pickle('./plot_df.pk')
grouped = plot_df.groupby(['Be_value','M'])

vcritmin_dict={}

for g,group in grouped: 
    x=group['M']
    y=group['V_critmin'].min(skipna=True)
    #print g, y
    vcritmin_dict[g[0], g[1]]= y
    
#print vcritmin_dict



In [ ]:
B_vals=[0.6,0.7,0.8,0.9,0.99]
v_crit_min_master={}
for B in B_vals[0:3]:
    crit_rot_data={'B':B}
    for M in mass_list[0:3]:
        message=str(B)+ '---'+ str(M)
        sys.stdout.write ('\r'+ message)
        
        v_minM=vcritmin_dict[B,M]
        v_maxM=vmax_dict[M]
        v_vals=tqdm(np.linspace(v_minM,v_maxM, 100))
        crit_times=map(lambda v: v_over_vcrit_interp(M, v, v_T_interp, False), v_vals)
        crit_rots_df=pd.DataFrame()
        for r in  crit_times:
            #first time star is faster than B
            crit_rotators=r[r['V/Vcrit']>B]

            if crit_rotators.empty == False:
                v_i=r['V_init'][0]
                t_1=r[r['V/Vcrit']>B]#.iloc[0]['t']
                #t_2=r[r['V/Vcrit']>B].iloc[-1]['t']
                #print 'V_i=',np.round(v_i,2),'t_1=' , np.round(t_1/1e6,3) , 't_2=', np.round( t_2/1e6,3)

                crit_rots_df=crit_rots_df.append(t_1)
                #crit_rots_df=crit_rots_df.append(pd.DataFrame([[M,v_i,t_1,t_2]], 
                #                                       columns=['M','V_init', 't1', 't2']))

        crit_rots_df.reset_index(drop=True, inplace=True)
        crit_rot_data[M]= crit_rots_df
        v_crit_min_master[B]= crit_rot_data
        
        #plt.plot( crit_rots_df['t']/1e6,crit_rots_df['V_init'], 'o')
        #plt.plot( crit_rots_df['t2']/1e6,crit_rots_df['V_init'], 'o')
        #plt.title(M)
        #plt.show()


#pickle.dump( crit_rot_data, open( "crit_rot_data.pk", "wb" ) )

In [ ]:
v_critmin_boundary={}
for M in mass_list[0:3]:#np.sort(v_crit_min_master[B].keys()):

    for B in np.sort(v_crit_min_master.keys()):
        crit_rots_df=v_crit_min_master[B][M]

        #crit_rots_df= crit_rot_data[M]
        vcritmin_df=pd.DataFrame()
        for vi in np.unique(crit_rots_df['V_init']):
            di=crit_rots_df[crit_rots_df['V_init'] == vi ]
            x=di['t']
            y=di['V_init']

            ix=x.argmax()
            im= x.argmin()

            #plt.plot(x.loc[ix], y.loc[ix], 'r.')
            #plt.plot(x.loc[im], y.loc[im], 'g.')

            
            if np.isnan(ix) == False:
                vcritmin_df=vcritmin_df.append(pd.DataFrame([[x.loc[ix], y.loc[ix], di['t/T'].loc[ix]]], 
                                                            columns=['t', 'Vcrit_min', 't/T']))
            if np.isnan(im) ==False :
                vcritmin_df=vcritmin_df.append(pd.DataFrame([[x.loc[im], y.loc[im], di['t/T'].loc[im]]], 
                                                            columns=['t', 'Vcrit_min', 't/T']))


        vcritmin_df=vcritmin_df.sort_values('t')
        vcritmin_df.reset_index(drop=True, inplace=True)
        
        v_critmin_boundary[B,M]= vcritmin_df
        
        x=vcritmin_df['t']
        y=vcritmin_df['Vcrit_min']
        plt.fill_between( x/1e6, y,crit_rots_df['V_init'].max(), alpha=0.8, label=B)
        #plt.plot(x,y, '-')
        plt.title(str(M) + str('$M_{\odot}$'))
        plt.legend(loc='lower left')
        plt.xlabel('t/Myr')
        plt.ylabel('V/$kms^{-1}$')
    plt.show()
    


In [ ]:
t=3e6
m_i=4.5
for t_frac in np.linspace(0,0.99,10):#np.linspace(0, M_V_T_interp(m_i, 400, v_T_interp), 1000):
    x_arr=[]
    y_arr=[]
    for M in mass_list[0:3]:
        dc=v_critmin_boundary[0.7,M]
        dc.reset_index(drop=True, inplace=True)
        ix=(dc['t/T']-t_frac).abs().argmin()
        x=M
        y=dc['Vcrit_min'].iloc[ix].min()
        #print dc.to_string()
        #print dc['t/T'].iloc[ix], t_frac, y
        x_arr=np.append(x_arr, x)
        y_arr=np.append(y_arr, y)


    straight_line1=interpolate.interp1d(x_arr, y_arr,
                                            bounds_error=True, assume_sorted=True)



    #plt.plot(x_arr,y_arr, 'ko-')
    #plt.plot(m_i, straight_line1(m_i), 'ro')
    V_minMt=straight_line1(m_i)
    plt.plot(t_frac, V_minMt, 'r.')
plt.title(m_i)
plt.show()


In [ ]:
s1=v_over_vcrit_interp(20, 200, v_T_interp, False)
s2=v_over_vcrit_interp(15, 200, v_T_interp, False)
print s1.columns.tolist()
plt.plot(s2['t/T'], s2['V/Vcrit'], label= s2['M'].iloc[0])
plt.plot(s1['t/T'], s1['V/Vcrit'], label= s1['M'].iloc[0])
plt.show()

In [ ]:
#plot V_critmin against m for a point in time 
#crit_rot_data= pickle.load('./crit_rot_data.pk')
t=5e6
def V_critmin_interp(t, m_i):
    m_V_critmin_df=pd.DataFrame()
    for m in np.sort(crit_rot_data.keys()):
        di= crit_rot_data[m]

        di.reset_index(drop=True, inplace=True)

        ti=((di['t1']-(t)).abs()).argmin()
        v_critmin= di.iloc[ti]['V_init']


        m_V_critmin_df=m_V_critmin_df.append(pd.DataFrame([[m,v_critmin]], columns=['M', 'V_critmin']))

        #ti=((di['t2']-(t)).abs()).argmin()
        #v_critmin= di.iloc[ti]['V_init']
        #plt.plot(m, v_critmin, 'go')

    straight_line1=interpolate.interp1d(x=m_V_critmin_df['M'], y=m_V_critmin_df['V_critmin'],
                                                       bounds_error=True, assume_sorted=True)

    m_arr=np.linspace(10,12,50)
    plt.plot(m_arr, straight_line1(m_arr))
    plt.plot(m_V_critmin_df['M'], m_V_critmin_df['V_critmin'], 'ko')  
    plt.show()
    
    #print straight_line1(14)
    
    return straight_line1(m_i)

print V_critmin_interp(11e6, 10.5)

In [ ]:
print m_V_critmin_df

In [3]:
a=v_over_vcrit_interp(12, 364) #12M initially at v/vcrit=0.5
b=v_over_vcrit_interp(15, 376) # initially at v/vcrit=0.5
c=v_over_vcrit_interp(20, 393)# initially at v/vcrit=0.5

plt.plot( a['t/T'],a['V/Vcrit'], 'r-', label=a.M[0])
plt.plot( b['t/T'],b['V/Vcrit'], 'k-', label=b.M[0])
plt.plot( c['t/T'],c['V/Vcrit'], label=c.M[0])
plt.legend()
plt.xlabel('t/T')
plt.ylabel('km/s')
plt.show()

TypeError: v_over_vcrit_interp() takes exactly 5 arguments (2 given)

In [ ]:

plot_df=pd.read_pickle('./plot_df.pk')




#print plot_df.head(100)

In [ ]:
grouped = plot_df.groupby(['Be_value','M'])

plot2=pd.DataFrame()

for g,group in grouped: 
    y_vals=[]
    #print group
    x=group['M']
    y=group['V_critmin'].min(skipna=True)
    y_vals=y_vals.append(y)
    plot2=plot2.append(pd.DataFrame([[g[1], y, g[0]]], columns=['M', 'V_critmin', 'Be_value']))
    
#print plot_df
#print plot2
grouped = plot2.groupby(['Be_value'])
for g,group in grouped: 
    x=group.M
    y=group['V_critmin']
    
    plt.plot(x,y,'o-', label=g)
    
    
plt.xlabel(r'$M_{\odot}$')
plt.ylabel(r'$V_{init}/kms^{-1}$')
plt.title('Min. $V_{init}$ for $V/V_{crit}$ to reach certain value')

plt.legend()
#plt.savefig('V_crit-min_plot4.png', dpi=300)
plt.show()



In [ ]:
interp_df=pd.DataFrame()
v_crit_interpolation={}
for m in  mass_list:
    dx=data_df[data_df.M ==m]
    for vi in np.unique(dx['V_init'])[1:]:
        dl=dx[(dx['V_init']==vi)]
        x=dl['V_crit'][0:10].mean()
        y=dl['V_init']
        #print m, vi,x
        #plt.plot(vi,x, 'o')
        interp_df=interp_df.append(pd.DataFrame([[m,vi,x]], columns=['M', 'V_init', 'V_crit_init']))
        
    #plt.title(m)    
    #plt.ylim(0,1.05)
    #plt.axhline(0.9)
    #plt.xlabel('V initial')
    #plt.ylabel('V_crit initial')
    #plt.show()
interp_df.head()

grouped = interp_df.groupby(['M'])
for g,group in grouped: 
    x=group['V_init']
    y=group['V_crit_init']
    straight_line1=interpolate.interp1d(x,y,
                                        bounds_error=False, assume_sorted=True)
    
    vs=np.linspace(0,600,100)
    v_crit_interpolation[g]= straight_line1
    plt.plot(vs, straight_line1(vs))
    plt.title(g)
    plt.show()
    
    
print v_crit_interpolation

In [ ]:

interp_vcrit= lambda x: v_crit_interpolation[x['M']](x['V_critmin'])
plot_df.reset_index(inplace=True)
for index, row in plot_df.iterrows():
    #print index
    #print v_crit_interpolation[row.M](row['V_critmin'])
    #row['Vcrit_init']= v_crit_interpolation[row.M](row['V_critmin'])
    plot_df.set_value(index,'Vcrit_init', v_crit_interpolation[row.M](row['V_critmin']))
    
print plot_df.to_string()

In [ ]:
grouped = plot_df.groupby(['Be_value','M'])

plot2=pd.DataFrame()

for g,group in grouped: 
    x=group['M']
    idx=group['V_critmin'].argmin(skipna=True)
    if np.isfinite(idx):
        Vi=group['V_critmin'].loc[idx]
        Vc=group['Vcrit_init'].loc[idx]
        y=Vi/Vc
    else:
        y=np.nan
        
    plot2=plot2.append(pd.DataFrame([[g[1], y, g[0]]], columns=['M', 'V/V_critmin', 'Be_value']))

#print plot_df
#print plot2.to_string()
grouped = plot2.groupby(['Be_value'])
for g,group in grouped: 
    x=group.M
    y=group['V/V_critmin']
    
    plt.plot(x,y,'o-', label=g)
    
    
plt.xlabel(r'$M_{\odot}$')
plt.ylabel(r'$V/V_{crit}$')
plt.title('Min. initial critical fraction for $V/V_{crit}$ to reach certain value')

plt.legend()
#plt.savefig('V_crit-min_plot3.png', dpi=300)
plt.show()


In [ ]:
for vi in np.unique(data_df['V'])[1:]:
    for m in  mass_list:
        dx=data_df[data_df.M ==m]
    
        dl=dx[(dx['V']==vi)]
        y=dl['V_crit'][0:10].mean()
        x=m
        #print m, vi,x
        plt.plot(x,y, 'o')
        
    plt.title(vi)    
    #plt.ylim(0,1.05)
    #plt.axhline(0.9)
    #plt.xlabel('V initial')
    #plt.ylabel('V_crit initial')
    plt.show()

In [ ]:
print data_df.head()

In [ ]:
#print MS_finder2('/home/ben/Documents/Bonn-uni/internship/takeaway_data/f10-50.mw.dat2')
print data_df.columns.tolist()
plt.clf()
for m in np.unique(data_df.M):
    
    dx=data_df[data_df.M ==m]
    for vi in np.unique(dx['V_init'])[1::4]:
        dl=dx[(dx['V_init']==vi) & (dx['t/Tms'] <0.99)]
        x=dl['t/Tms']
        y=dl['2:M/Msun'] *dl['5:R/Rsun']**2 #  /dl['2:M/Msun'].iloc[0]
        plt.plot(x,y, label=str(int(vi)) +' km/s')
        plt.title(m)    
        
    #plt.ylim(0,1.05)
    plt.xlabel(r'$t/T_{MS}$')
    plt.ylabel(r'$I$')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),ncol=5)
    
    plt.savefig('/home/ben/Documents/Bonn-uni/internship/plots/I/'+str(m)+'.png', dpi=300,bbox_inches='tight')
    plt.show()
    plt.clf()